<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Sebastián Reyes &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [21]:
# Paquetes necesarios
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import joblib  # Para guardar el modelo

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [8]:
# Descargar el conjunto de datos
path = kagglehub.dataset_download("imakash3011/customer-personality-analysis")

# Cargar el conjunto de datos
df = pd.read_csv(f"{path}/marketing_campaign.csv", sep="\t")

# Exploración de Datos
print("🔍 Exploración Inicial de Datos:\n")

# Primeros registros con display
print("📊 Primeros 5 Registros:")
display(df.head())

🔍 Exploración Inicial de Datos:

📊 Primeros 5 Registros:


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [12]:
# Selección de variables relevantes
variables = [
    "Income", "MntWines", "MntFruits", "MntMeatProducts",
    "MntFishProducts", "MntSweetProducts", "MntGoldProds",
    "NumDealsPurchases", "NumWebPurchases", "NumCatalogPurchases",
    "NumStorePurchases", "NumWebVisitsMonth"
]

target = "Response"

# Limpieza inicial: Eliminación de filas con valores faltantes
df = df.dropna(subset=variables + [target])

# División en características y etiquetas
X = df[variables]
y = df[target]

# Escalamiento de las variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División del conjunto en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

Tamaño del conjunto de entrenamiento: (1772, 12)
Tamaño del conjunto de prueba: (444, 12)



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [15]:
# Modelo inicial con k=9
knn_optimal = KNeighborsClassifier(n_neighbors=9)
knn_optimal.fit(X_train, y_train)
y_test_pred = knn_optimal.predict(X_test)

# Mostrar métricas del modelo
print("📊 Métricas del modelo inicial:")
print(f"Precisión: {accuracy_score(y_test, y_test_pred):.2f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_test_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_test_pred))

📊 Métricas del modelo inicial:
Precisión: 0.87

Matriz de Confusión:
[[376   1]
 [ 57  10]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       377
           1       0.91      0.15      0.26        67

    accuracy                           0.87       444
   macro avg       0.89      0.57      0.59       444
weighted avg       0.87      0.87      0.83       444



# Optimización de Hiperparámetros con GridSearch

## Significado de 4 Hiperparámetros del Modelo k-NN

1. **`n_neighbors`**:
   - Especifica el número de vecinos más cercanos que el modelo considerará para tomar una decisión de clasificación o regresión.
   - Un valor bajo hace que el modelo sea más sensible al ruido, mientras que uno alto genera un modelo más generalizado.

2. **`weights`**:
   - Determina cómo se ponderan los vecinos para la decisión final.
     - `'uniform'`: Cada vecino contribuye de manera equitativa.
     - `'distance'`: Los vecinos más cercanos tienen mayor peso.
     - Función personalizada: Se puede definir una regla específica de ponderación.

3. **`metric`**:
   - Especifica la fórmula matemática que se utiliza para calcular la distancia entre puntos. Algunos valores comunes son:
     - `'euclidean'`: Distancia euclidiana.
     - `'manhattan'`: Distancia de Manhattan (suma de diferencias absolutas).
     - `'minkowski'`: Generalización de ambas distancias con un parámetro \( p \).

4. **`p`**:
   - Es un hiperparámetro asociado con la métrica `'minkowski'`.
     - \( p = 1 \): Distancia de Manhattan.
     - \( p = 2 \): Distancia euclidiana.
   - Define el grado de la norma utilizada en el cálculo de la distancia.

---

## Hiperparámetros Seleccionados para Optimizar

Para optimizar el modelo k-NN, los siguientes hiperparámetros serán seleccionados:

1. **`n_neighbors`**:
   - Valores a probar: `[5, 7, 9, 11, 13]`.

2. **`weights`**:
   - Métodos de ponderación: `['uniform', 'distance']`.

3. **`metric`**:
   - Métricas de distancia: `['euclidean', 'manhattan', 'minkowski']`.

4. **`p`**:
   - Solo se aplica cuando se utiliza `'minkowski'`.
   - Valores a probar: `[1, 2]`.




### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [17]:
# Configuración de hiperparámetros para Grid Search
param_grid = {
    "n_neighbors": [5, 7, 9, 11, 13],  # Valores para el número de vecinos
    "weights": ["uniform", "distance"],  # Métodos de ponderación
    "metric": ["euclidean", "manhattan", "minkowski"],  # Distancias
    "p": [1, 2]  # Parámetro p para 'minkowski' (1: Manhattan, 2: Euclidiana)
}

# Configurar y ejecutar Grid Search con 5 validaciones cruzadas
grid_search = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,  # Número de validaciones cruzadas
    verbose=1,
    n_jobs=-1  # Usar todos los núcleos disponibles para acelerar
)

grid_search.fit(X_train, y_train)

# Mostrar resultados de la optimización
print("\nMejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print(f"\nMejor precisión en validación cruzada: {grid_search.best_score_:.2f}")

# Evaluar el modelo optimizado en el conjunto de prueba
best_model = grid_search.best_estimator_
y_test_pred_optimized = best_model.predict(X_test)

# Mostrar métricas del modelo optimizado
print("\n📊 Métricas del modelo optimizado en el conjunto de prueba:")
print(f"Precisión: {accuracy_score(y_test, y_test_pred_optimized):.2f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_test_pred_optimized))

Fitting 5 folds for each of 60 candidates, totalling 300 fits

Mejores hiperparámetros encontrados:
{'metric': 'manhattan', 'n_neighbors': 13, 'p': 1, 'weights': 'distance'}

Mejor precisión en validación cruzada: 0.86

📊 Métricas del modelo optimizado en el conjunto de prueba:
Precisión: 0.87

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       377
           1       0.79      0.16      0.27        67

    accuracy                           0.87       444
   macro avg       0.83      0.58      0.60       444
weighted avg       0.86      0.87      0.83       444



### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [20]:
# Configuración de hiperparámetros para RandomizedSearchCV
param_distributions = {
    "n_neighbors": [3, 5, 7, 9, 11, 13, 15],  # Valores para el número de vecinos
    "weights": ["uniform", "distance"],  # Métodos de ponderación
    "metric": ["euclidean", "manhattan", "minkowski"],  # Distancias
    "p": [1, 2, 3, 4, 5]  # Parámetro p para 'minkowski'
}

# Configurar y ejecutar RandomizedSearchCV con 25 iteraciones y 5 validaciones cruzadas
random_search = RandomizedSearchCV(
    estimator=KNeighborsClassifier(),
    param_distributions=param_distributions,
    n_iter=25,  # Número de combinaciones a probar
    scoring="accuracy",
    cv=5,  # Número de validaciones cruzadas
    verbose=1,
    random_state=42,
    n_jobs=-1  # Usar todos los núcleos disponibles para acelerar
)

random_search.fit(X_train, y_train)

# Mostrar resultados de la optimización
print("\nMejores hiperparámetros encontrados:")
print(random_search.best_params_)

print(f"\nMejor precisión en validación cruzada: {random_search.best_score_:.2f}")

# Evaluar el modelo optimizado en el conjunto de prueba
best_model = random_search.best_estimator_
y_test_pred_optimized = best_model.predict(X_test)

# Mostrar métricas del modelo optimizado
print("\n📊 Métricas del modelo optimizado en el conjunto de prueba:")
print(f"Precisión: {accuracy_score(y_test, y_test_pred_optimized):.2f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_test_pred_optimized))

Fitting 5 folds for each of 25 candidates, totalling 125 fits

Mejores hiperparámetros encontrados:
{'weights': 'distance', 'p': 4, 'n_neighbors': 15, 'metric': 'minkowski'}

Mejor precisión en validación cruzada: 0.86

📊 Métricas del modelo optimizado en el conjunto de prueba:
Precisión: 0.87

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       377
           1       0.83      0.15      0.25        67

    accuracy                           0.87       444
   macro avg       0.85      0.57      0.59       444
weighted avg       0.86      0.87      0.83       444



### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [22]:
# Configuración del modelo más simple
simple_model_params = {'metric': 'manhattan', 'n_neighbors': 13, 'p': 1, 'weights': 'distance'}

# Crear y entrenar el modelo con los parámetros más simples
simple_knn_model = KNeighborsClassifier(
    n_neighbors=simple_model_params['n_neighbors'],
    weights=simple_model_params['weights'],
    metric=simple_model_params['metric'],
    p=simple_model_params['p']
)

# Entrenamiento con todo el conjunto de entrenamiento
simple_knn_model.fit(X_train, y_train)

# Evaluación en el conjunto de prueba
y_test_pred_simple = simple_knn_model.predict(X_test)

# Mostrar resultados del modelo
print("\n📊 Métricas del modelo más simple en el conjunto de prueba:")
print(f"Precisión: {accuracy_score(y_test, y_test_pred_simple):.2f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_test_pred_simple))


📊 Métricas del modelo más simple en el conjunto de prueba:
Precisión: 0.87

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       377
           1       0.79      0.16      0.27        67

    accuracy                           0.87       444
   macro avg       0.83      0.58      0.60       444
weighted avg       0.86      0.87      0.83       444



In [23]:
# Guardar el modelo entrenado
simple_model_filename = "simple_knn_model.pkl"
joblib.dump(simple_knn_model, simple_model_filename)
print(f"\nModelo más simple guardado como '{simple_model_filename}'.")


Modelo más simple guardado como 'simple_knn_model.pkl'.


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>